<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/PreTrained_LCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
!unzip "/content/gdrive/MyDrive/lfw_patch.zip" -d "/content/" 

Streaming output truncated to the last 5000 lines.
  inflating: /content/lfw_patch/Pak_Gil_Yon/Pak_Gil_Yon_0001.jpg  
   creating: /content/lfw_patch/Pamela_Anderson/
  inflating: /content/lfw_patch/Pamela_Anderson/Pamela_Anderson_0001.jpg  
  inflating: /content/lfw_patch/Pamela_Anderson/Pamela_Anderson_0002.jpg  
  inflating: /content/lfw_patch/Pamela_Anderson/Pamela_Anderson_0003.jpg  
  inflating: /content/lfw_patch/Pamela_Anderson/Pamela_Anderson_0004.jpg  
  inflating: /content/lfw_patch/Pamela_Anderson/Pamela_Anderson_0005.jpg  
   creating: /content/lfw_patch/Pamela_Melroy/
  inflating: /content/lfw_patch/Pamela_Melroy/Pamela_Melroy_0001.jpg  
   creating: /content/lfw_patch/Paola_Espinoza/
  inflating: /content/lfw_patch/Paola_Espinoza/Paola_Espinoza_0001.jpg  
   creating: /content/lfw_patch/Paradorn_Srichaphan/
  inflating: /content/lfw_patch/Paradorn_Srichaphan/Paradorn_Srichaphan_0001.jpg  
  inflating: /content/lfw_patch/Paradorn_Srichaphan/Paradorn_Srichaphan_0002.jpg  


In [4]:
import torch.utils.data as data

from PIL import Image
import os
import os.path

def default_loader(path):
    img = Image.open(path).convert('L')
    return img

def default_list_reader(fileList):
    imgList = []
    with open(fileList, 'r') as file:
        for line in file.readlines():
            imgPath, label = line.strip().split(' ')
            imgList.append((imgPath, int(label)))
    return imgList

class ImageList(data.Dataset):
    def __init__(self, root, fileList, transform=None, list_reader=default_list_reader, loader=default_loader):
        self.root      = root
        self.imgList   = list_reader(fileList)
        self.transform = transform
        self.loader    = loader

    def __getitem__(self, index):
        imgPath, target = self.imgList[index]
        img = self.loader(os.path.join(self.root, imgPath))

        if self.transform is not None:
            img = self.transform(img)
        return img, target

    def __len__(self):
        return len(self.imgList)

In [5]:
'''
    implement Light CNN
    @author: Alfred Xiang Wu
    @date: 2017.07.04
'''

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class mfm(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, type=1):
        super(mfm, self).__init__()
        self.out_channels = out_channels
        if type == 1:
            self.filter = nn.Conv2d(in_channels, 2*out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        else:
            self.filter = nn.Linear(in_channels, 2*out_channels)

    def forward(self, x):
        x = self.filter(x)
        out = torch.split(x, self.out_channels, 1)
        return torch.max(out[0], out[1])

class group(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(group, self).__init__()
        self.conv_a = mfm(in_channels, in_channels, 1, 1, 0)
        self.conv   = mfm(in_channels, out_channels, kernel_size, stride, padding)

    def forward(self, x):
        x = self.conv_a(x)
        x = self.conv(x)
        return x

class resblock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(resblock, self).__init__()
        self.conv1 = mfm(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = mfm(in_channels, out_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        res = x
        out = self.conv1(x)
        out = self.conv2(out)
        out = out + res
        return out

     

class network_29layers(nn.Module):
    def __init__(self, block, layers, num_classes=79077):
        super(network_29layers, self).__init__()
        self.conv1  = mfm(1, 48, 5, 1, 2)
        self.pool1  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.block1 = self._make_layer(block, layers[0], 48, 48)
        self.group1 = group(48, 96, 3, 1, 1)
        self.pool2  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.block2 = self._make_layer(block, layers[1], 96, 96)
        self.group2 = group(96, 192, 3, 1, 1)
        self.pool3  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.block3 = self._make_layer(block, layers[2], 192, 192)
        self.group3 = group(192, 128, 3, 1, 1)
        self.block4 = self._make_layer(block, layers[3], 128, 128)
        self.group4 = group(128, 128, 3, 1, 1)
        self.pool4  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.fc     = mfm(8*8*128, 256, type=0)
        self.fc2    = nn.Linear(256, num_classes)
            
    def _make_layer(self, block, num_blocks, in_channels, out_channels):
        layers = []
        for i in range(0, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)

        x = self.block1(x)
        x = self.group1(x)
        x = self.pool2(x)

        x = self.block2(x)
        x = self.group2(x)
        x = self.pool3(x)

        x = self.block3(x)
        x = self.group3(x)
        x = self.block4(x)
        x = self.group4(x)
        x = self.pool4(x)

        x = x.view(x.size(0), -1)
        fc = self.fc(x)
        fc = F.dropout(fc, training=self.training)
        out = self.fc2(fc)
        return out, fc


def LightCNN_29Layers(**kwargs):
    model = network_29layers(resblock, [1, 2, 3, 4], **kwargs)
    return model



In [6]:
model_path = "/content/gdrive/MyDrive/LightCNN_29Layers_checkpoint.pth.tar"
dataset = "/content/lfw_patch"
imglist = "/content/gdrive/MyDrive/imglist.txt"
save_path = "/content/data"

In [7]:

'''
    implement the feature extractions for light CNN
    @author: Alfred Xiang Wu
    @date: 2017.07.04
'''

from __future__ import print_function
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import numpy as np
import cv2




def main():
    model = LightCNN_29Layers(num_classes=79077)

    model.eval()
    model = torch.nn.DataParallel(model).cuda()


    if os.path.isfile(model_path):
        print("=> loading checkpoint '{}'".format(model_path))
        checkpoint = torch.load(model_path)
        model.load_state_dict(checkpoint['state_dict'])

    img_path_list  = create_img_path(imglist)
    transform = transforms.Compose([transforms.ToTensor()])
    count     = 0
    input     = torch.zeros(1, 1, 128, 128)
    for img_name in img_path_list:
        count = count + 1
       
        img   = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
        
        if img.shape != (128, 128):
            img = cv2.resize(img, (128, 128))
        

        img   = np.reshape(img, (128, 128, 1))
        img   = transform(img)
        input[0,:,:,:] = img

        start = time.time()
        
        input = input.cuda()
        input_var   = torch.autograd.Variable(input, volatile=True)
        _, features = model(input_var)

        print(features.data.cpu().numpy()[0])


def read_list(list_path):
    img_list = []
    with open(list_path, 'r') as f:
        for line in f.readlines()[1:]:

            img_path = line.strip().split()
            img_list.append(img_path[0])
            
    return img_list


def create_img_path(imglist):
  img_list = read_list(imglist)
  img_path_list = []
  for item in img_list:
    folder_path = os.path.join(dataset, item)
    for img in os.listdir(folder_path):
      img_path = os.path.join(folder_path, img)
      
      img_path_list.append(img_path)

  return img_path_list


if __name__ == '__main__':
    main()

=> loading checkpoint '/content/gdrive/MyDrive/LightCNN_29Layers_checkpoint.pth.tar'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Streaming output truncated to the last 5000 lines.
  2.47536564e+00 -8.42663479e+00  7.65937507e-01  1.12958384e+01
  4.86771917e+00 -6.08690786e+00 -1.78390331e+01 -8.34994698e+00
 -2.73981495e+01  2.83574734e+01 -3.33414688e+01 -1.20869875e+01
 -3.79147224e+01  2.16267643e+01  1.57435303e+01 -6.52491903e+00
  1.02938938e+01  5.74211216e+00  4.10675888e+01  2.83654289e+01
 -7.58843040e+00 -2.00590387e-01 -4.52285576e+00 -2.15768356e+01
 -1.46147699e+01 -3.48129959e+01  2.01374645e+01 -7.56937790e+00
  3.81713028e+01 -1.25162420e+01 -2.68680553e+01  6.63049555e+00
  8.63317871e+00  3.95735335e+00 -2.30612717e+01 -1.73226185e-03
  1.21212749e+01 -8.27841610e-02 -1.79429398e+01  1.30604420e+01
 -1.99519844e+01  1.68751640e+01  7.33333302e+00  3.88382721e+01
  8.93108463e+00 -1.47805843e+01  6.52957106e+00  5.17334366e+00]
[-36.407463     4.4943295   -2.9180493   18.712019    12.9720545
  34.064907    14.408102    -8.609673     2.4171114    3.0417
  25.214449   -36.275753    38.212086    

In [36]:
def calc_hist(path):

      model = LightCNN_29Layers(num_classes=79077)

      model.eval()
      model = torch.nn.DataParallel(model).cuda()


      transform = transforms.Compose([transforms.ToTensor()])
      input     = torch.zeros(1, 1, 128, 128)

      if os.path.isfile(model_path):
        print("=> loading checkpoint '{}'".format(model_path))
        checkpoint = torch.load(model_path)
        model.load_state_dict(checkpoint['state_dict'])

      img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
      if img.shape != (128, 128):
            img = cv2.resize(img, (128, 128))
      img   = np.reshape(img, (128, 128, 1))
      
      img   = transform(img)
      input[0,:,:,:] = img

      start = time.time()
        
      input = input.cuda()
      input_var   = torch.autograd.Variable(input, volatile=True)
      _, features = model(input_var)
      #imgLBP    = getLBPimage(img_path)
      flattened_feature = features.flatten()
      flattened_feature = flattened_feature.cpu()
      flattened_feature = flattened_feature.detach().numpy()
      print(flattened_feature)
      hist = cv2.calcHist([flattened_feature], [0], None, [256], [0, 256])
    
      return hist

In [37]:
calc_hist('/content/lfw_patch/Aaron_Sorkin/Aaron_Sorkin_0001.jpg')

=> loading checkpoint '/content/gdrive/MyDrive/LightCNN_29Layers_checkpoint.pth.tar'
[ 24.130116    -1.8112737   -5.912782    18.95601     12.171851
 -17.876144   -24.422844     7.4050474    3.876483   -10.1723585
 -17.092602     4.2853513  -12.869367    16.696356    -4.9053864
  -6.445178    33.65403      4.0610514   14.392281   -10.47466
  15.7100725   25.84529    -15.676827    21.248045     5.9261284
   9.121889   -27.724718    33.577507     1.3521833    0.23426849
 -18.334772     3.8261335   -3.7929327   -3.1784313    5.451367
  22.77711     12.503713    -4.629497   -12.020446    -6.3708954
 -20.375082    -8.759702   -22.142052     1.401629   -12.676921
  -9.82909     47.984375    14.755338    -0.726224    -2.6244357
 -13.571152    -9.359163    16.523659    -2.7214026   -7.604795
 -17.963572    32.45058    -26.089176     5.0196795  -40.373096
  -6.684655    18.700521    -0.5452181  -11.840361    -5.519596
   8.957888    10.801613    12.372004     2.1803093   -7.2615404
  10.240899 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


array([[5.],
       [6.],
       [3.],
       [8.],
       [6.],
       [8.],
       [5.],
       [3.],
       [4.],
       [6.],
       [4.],
       [6.],
       [7.],
       [2.],
       [4.],
       [3.],
       [5.],
       [1.],
       [3.],
       [6.],
       [1.],
       [2.],
       [4.],
       [4.],
       [1.],
       [4.],
       [2.],
       [1.],
       [4.],
       [0.],
       [1.],
       [0.],
       [2.],
       [2.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [38]:
def eucledean_dist_hist(hist1, hist2):
  sum = 0
  for i in range (0,256):
    sum = sum + (hist1[i][0]-hist2[i][0])**2
  dist = math.sqrt(sum)
  print('euclidean distance:', dist)

In [39]:
hist1 = hist1 = calc_hist('/content/lfw_patch/Aaron_Peirsol/Aaron_Peirsol_0003.jpg')
hist2 = calc_hist('/content/lfw_patch/Abbas_Kiarostami/Abbas_Kiarostami_0001.jpg')

=> loading checkpoint '/content/gdrive/MyDrive/LightCNN_29Layers_checkpoint.pth.tar'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


[ 2.27187586e+00 -1.75314516e-01  2.86672616e+00 -6.92375600e-01
  1.02810621e+01 -6.97072363e+00 -9.52495670e+00  1.80726891e+01
 -1.54683371e+01  1.15373678e+01  8.34766102e+00 -9.39328861e+00
 -2.45932937e+00 -7.99630821e-01  2.41868019e+00  2.70899639e+01
  1.41831722e+01 -1.25298920e+01  1.07196150e+01 -5.64375401e+00
  9.48014259e+00 -5.30111611e-01  1.02643528e+01 -2.45308456e+01
 -1.64340057e+01  2.68878918e+01  1.83749199e+00  2.31100883e+01
 -1.49270840e+01 -8.17424583e+00 -4.47516966e+00 -1.33264184e+00
  1.71528683e+01  1.10908470e+01 -3.23793769e+00 -1.02684488e+01
  1.99314651e+01  3.02524700e+01 -1.11896620e+01  1.40056229e+01
  2.24145908e+01 -2.24778652e+01  1.24373922e+01  1.46029723e+00
  2.41298885e+01  7.95649195e+00  5.65681505e+00 -9.35302377e-01
 -1.21357667e+00  3.13718872e+01  2.44019146e+01  7.40700841e-01
  1.43286295e+01  1.58084335e+01 -7.37477875e+00 -7.96385670e+00
 -1.81402626e+01 -2.11205254e+01  2.63184147e+01 -1.31657000e+01
 -2.00186386e+01  1.39019

In [40]:
eucledean_dist_hist(hist1, hist2)

euclidean distance: 13.892443989449804
